## Yaw alignment based on the position of the face

In [7]:
import cv2
import numpy as np

w, h = 360, 240
fbRange = [6200, 6800]
pid = [0.4, 0.4, 0]
pError = 0

In [2]:
def findFace(img):
    faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(imgGray, 1.2, 8)

    myFaceListC = []               # info of center point
    myFaceListArea = []

    for (x,y,w,h) in faces:
        cv2.rectangle(img, (x,y), (x+w, y+h), (0,0,255), 2)
        cx = x + w//2
        cy = y + h//2
        area = w * h
        cv2.circle(img, (cx, cy), 5, (0,255,0), cv2.FILLED)
        myFaceListC.append([cx, cy])
        myFaceListArea.append(area)

    if len(myFaceListArea) != 0:
        i = myFaceListArea.index(max(myFaceListArea))
        return img, [myFaceListC[i], myFaceListArea[i]]
    else:
        return img, [[0,0], 0]

In [3]:
import cvlib

def findFaceResnet(img):
    
    # Perform face detection
    faces, confidences = cvlib.detect_face(img)

    myFaceListC = []               # info of center point
    myFaceListArea = []

    for (x1,y1,x2,y2) in faces:
        cv2.rectangle(img, (x1,y1), (x2, y2), (0,0,255), 2)
        cx = (x1 + x2)//2
        cy = (y1 + y2)//2
        area = (x2-x1) * (y2-y1)
        cv2.circle(img, (cx, cy), 5, (0,255,0), cv2.FILLED)
        myFaceListC.append([cx, cy])
        myFaceListArea.append(area)

    if len(myFaceListArea) != 0:
        i = myFaceListArea.index(max(myFaceListArea))
        return img, [myFaceListC[i], myFaceListArea[i]]
    else:
        return img, [[0,0], 0]

In [4]:
# def trackFace(me, info, w, pid, pError):
def trackFace(info, w, pid, pError):
    area = info[1]
    x, y = info[0]
    fb = 0
    
    # pid for the angle
    error = x - w//2    # w is width of the image, x is face center
    speed = pid[0]*error + pid[1]*(error - pError)
    speed = int(np.clip(speed, -100, 100))    # clip the speed between -100 and 100
    
    
    # Forward backward movement based on area of bounding box
    if area > fbRange[0] and area < fbRange[1]:
        fb = 0
    elif area > fbRange[1]:
        fb=-20    # go back
    elif area < fbRange[0] and area != 0:
        fb = 20   # go forward

    if x == 0:
        speed = 0
        error = 0
    
    # print(speed, fb)
    # send commands to drone
    # me.send_rc_control(0, fb, 0, speed)    # 4th one is yaw which is speed
    return error, speed, fb

In [9]:
cap = cv2.VideoCapture(1)

while True:
    _, img = cap.read()
    img = cv2.resize(img, (w, h))
    img, info = findFace(img)
    # pError = trackFace(me, info, w, pid, pError)
    pError, speed, fb = trackFace(info, w, pid, pError)
    # print("Center:", info[0], "Area:", info[1])
    cv2.putText(img, f'yaw: {speed}, fb: {fb}', (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
    cv2.imshow("Output", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [21]:
cap.release()
cv2.destroyAllWindows()